In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
import pickle

In [2]:
device = torch.device('cuda')
device

device(type='cuda')

# Load Data

In [3]:
from torch.utils.data import Dataset


def binary(x, bits):
    mask = 2**torch.arange(bits).to(x.device, x.dtype)
    return x.unsqueeze(-1).bitwise_and(mask).ne(0).float()


class VariableLenDataset(Dataset):
    def __init__(self, data, targets):
        self.X = [torch.tensor(x, dtype=torch.float32) for x in data]
        self.y = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [4]:
train_transform = transforms.Compose(
    [transforms.ToTensor()])

In [5]:
with open("data/train.pkl", "rb") as train_file:
    dataset = pickle.load(train_file)

In [6]:
df = pd.DataFrame(dataset)
df

,0,1
0,"[-1.0, -1.0, -1.0, -1.0, 144.0, 144.0, 144.0, ...",0
1,"[-1.0, -1.0, 144.0, 144.0, 34.0, 0.0, 4.0, 2.0...",0
2,"[66.0, 100.0, 148.0, 148.0, 146.0, 64.0, 146.0...",0
3,"[147.0, 65.0, 64.0, 146.0, 8.0, 50.0, 160.0, 5...",0
4,"[144.0, 144.0, 144.0, 144.0, 144.0, 64.0, 64.0...",0
...,...,...
2934,"[144.0, 144.0, 144.0, 66.0, 82.0, 132.0, 8.0, ...",4
2935,"[32.0, 33.0, 162.0, 121.0, 68.0, 68.0, 120.0, ...",4
2936,"[33.0, 33.0, 33.0, 33.0, 78.0, 2.0, 68.0, 34.0...",4
2937,"[96.0, 96.0, 96.0, 96.0, 176.0, 176.0, 176.0, ...",4


In [7]:
train_dataset = VariableLenDataset(df[0].values, df[1].values)

In [8]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

pad = 0

def pad_collate(batch, pad_value=0):
    xx, yy = zip(*batch)
    x_lens = [len(x) if x.dim() > 0 else 0 for x in xx]
    y_lens = [len(y) if y.dim() > 0 else 0 for y in yy]

    xx = [x.unsqueeze(0) if x.dim() == 0 else x for x in xx]
    yy = [y.unsqueeze(0) if y.dim() == 0 else y for y in yy]

    xx_pad = pad_sequence(xx, batch_first=False, padding_value=pad_value)
    yy_pad = pad_sequence(yy, batch_first=False, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

In [9]:
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True, collate_fn=pad_collate)

In [10]:
for a, b, c, d in train_loader:
    print(a)
    print(b)
    break

tensor([[ -1.,  33.,  64.,  ...,  -1., 112.,  65.],
        [102.,  33.,  64.,  ...,  -1., 112.,  65.],
        [143.,  33.,  64.,  ...,  -1.,   0.,  92.],
        ...,
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
        [  0.,   0.,   0.,  ...,   0.,   0.,   0.]])
tensor([[2, 0, 4, 0, 0, 0, 1, 0, 0, 4, 0, 0, 1, 0, 4, 0, 0, 0, 2, 3, 0, 3, 0, 0,
         0, 4, 2, 0, 0, 0, 3, 1, 1, 0, 3, 0, 4, 0, 0, 0, 3, 3, 0, 3, 4, 0, 0, 0,
         4, 3]])


# Model

In [38]:
class ComposerClasifier(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
        self.fc = nn.Linear(hidden_size, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, hidden):
        # x = torch.transpose(x,0,1)
        # print(x.shape)
        all_outputs, hidden = self.lstm(x, hidden)
        # all_outputs = torch.transpose(all_outputs,0,1)
        # out = torch.flatten(all_outputs,1)
        # print(all_outputs.shape)
        # all_outputs = torch.transpose(all_outputs,0,1)
        x = self.fc(all_outputs[-1])
        return x, hidden


In [72]:
hidden_dim = 32
layer_dim = 3
input_dim = 1
out_dim = df[1].nunique()
model = ComposerClasifier(input_dim, hidden_dim, layer_dim, out_dim)
model.to(device)

ComposerClasifier(
  (lstm): LSTM(1, 32, num_layers=3, dropout=0.4)
  (fc): Linear(in_features=32, out_features=5, bias=True)
)

In [73]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
loss_fun = nn.CrossEntropyLoss(torch.tensor([1.0, 2.0, 2.0, 4.0, 4.5]).to(device))

In [74]:
# Training loop
for epoch in range(101):
    loss_sum = 0
    for x, targets, x_lens, target_lens in train_loader:
        x = x.to(device).unsqueeze(2)
        # x = torch.transpose(x,0,1)
        targets = torch.transpose(targets,0,1)
        targets = targets.to(device).squeeze(1)

        hidden, state = model.init_hidden(x.size(1))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = model(x, (hidden,state))
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        loss_sum += loss.item()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss_sum:.3}")

Epoch: 0, loss: 90.0
Epoch: 10, loss: 88.1
Epoch: 20, loss: 86.9
Epoch: 30, loss: 87.9
Epoch: 40, loss: 85.7
Epoch: 50, loss: 87.1
Epoch: 60, loss: 85.7
Epoch: 70, loss: 85.3
Epoch: 80, loss: 83.3
Epoch: 90, loss: 84.4
Epoch: 100, loss: 85.4


In [75]:
total = 0
corect = 0
with torch.no_grad():
    for x, targets, x_len, target_len in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.shape[1])
        hidden, state = hidden.to(device), state.to(device)

#         x = torch.transpose(x, 0, 1)
#         preds, _ = model(x, (hidden, state))
#         preds = torch.transpose(preds, 0, 1)

        # x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        preds, _ = model(x, (hidden, state))
        preds = torch.argmax(preds, dim=1)
        # preds, pred_len = pad_packed_sequence(preds_packed, batch_first=True, padding_value=pad)
        for pred, targ in zip(preds, targets[0]):
            total += 1
            if torch.equal(pred, targ):
                corect += 1
        # preds = preds.squeeze(2)
        mask_tgt = targets != pad
        print(targets)
        print(preds)
        # # print(torch.abs(preds[mask_tgt] - targets[mask_tgt]).mean())
        # print()
print(corect/total)

tensor([[0, 4, 0, 0, 0, 2, 0, 0, 3, 1, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
         3, 1, 0, 3, 0, 0, 0, 2, 3, 1, 3, 0, 0, 4, 0, 0, 0, 0, 0, 3, 0, 1, 1, 0,
         0, 0]], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 3, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 3, 0, 0, 0, 3, 1, 0, 0,
        0, 0], device='cuda:0')
tensor([[0, 4, 0, 0, 1, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4, 3, 0, 0, 0, 2, 0, 0,
         0, 0, 3, 3, 0, 0, 3, 2, 0, 4, 1, 1, 0, 2, 1, 0, 1, 0, 3, 4, 0, 4, 1, 0,
         0, 1]], device='cuda:0')
tensor([3, 3, 0, 1, 0, 3, 0, 3, 0, 0, 3, 0, 0, 0, 1, 0, 1, 0, 0, 3, 0, 3, 0, 0,
        3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 0,
        0, 0], device='cuda:0')
tensor([[1, 0, 3, 1, 0, 0, 1, 0, 0, 0, 0, 3, 1, 0, 4, 0, 1, 0, 0, 1, 0, 1, 3, 3,
         2, 0, 3, 4, 1, 0, 1, 0, 0, 4, 4, 0, 2, 3, 1, 3, 1, 1, 0, 0, 0, 0, 3, 3,
         1, 0]], device='cuda:0')
tensor([3, 0, 0, 0, 0, 3, 1,